In [ ]:
# %pip install matplotlib pandas

In [ ]:
import glob
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:

# Get the name of latest CSV file
latest_csv_filename = sorted(glob.glob("./result-*.csv"))[-1]

csv_filenames = [ latest_csv_filename ]

# csv_filenames = [
#     "./result-20250111-004248.csv",
#     "./result-20250111-024434.csv",
#     "./result-20250111-223819.csv"
# ]

df_parts = []
for csv_filename in csv_filenames:
    df_parts.append( pd.read_csv(csv_filename) )

df = pd.concat(df_parts)

df

In [ ]:
# Add integer version of transfer_size column
def size_str_to_int(s):
    if s.endswith("K"):
        i = int(s[:-1])
    elif s.endswith("M"):
        i = int(s[:-1]) * 1024
    elif s.endswith("G"):
        i = int(s[:-1]) * 1024 * 1024
    elif s.endswith("T"):
        i = int(s[:-1]) * 1024 * 1024 * 1024
    else:
        assert False
    return i

df["transfer_size_int"] = df["transfer_size"].map(size_str_to_int)

df.head()

In [ ]:
df.dtypes

In [ ]:
# Common functions for plot

def format_num_processes(x, _):
    return "%d" % (x)

def format_filesize(x, _):
    if x<0:
        return str(x)
    for x_unit in ['KB', 'MB', 'GB', 'TB']:
        if x < 1024:
            return "%d%s" % (x, x_unit)
        x /= 1024.0

def format_bandwidth(y, _):
    return "%dMB/s" % (y)


In [ ]:
# test-1: how performance changes by number of processes and access granularity

xaxis_ticks = []
for i in range(10):
    xaxis_ticks.append( 1 * (2**i) )

def plot_graph( title, condition, values ):

    df2 = df[condition]
    df3 = df2.pivot(index="num_processes", columns=["filesystem_type"], values=values)
    if len(df3) == 0: return
    ax = df3.plot(title=title, logx=True, figsize=(6, 4), rot=0)
    ax.set_xlabel("num processes (log scale)")
    ax.xaxis.set_ticks(xaxis_ticks)
    ax.xaxis.set_major_formatter(format_num_processes)
    ax.yaxis.set_major_formatter(format_bandwidth)

for transfer_size in ["4K", "64K", "1M", "16M", "256M"]:
    plot_graph( f"Read bandwidth {transfer_size}", df["transfer_size"]==transfer_size, ["read_bw_mean"] )
    plot_graph( f"Write bandwidth {transfer_size}", df["transfer_size"]==transfer_size, ["write_bw_mean"] )
